In [1]:
import requests 
import pandas as pd    
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os


In [2]:
load_dotenv(dotenv_path='../erdos_finance_llm\erdos.env')

def get_news(symbols, start_date, end_date,api_key):
    if api_key is None:
        api_key = os.getenv('ALPHA_VANTAGE_API_KEY')
        if api_key is None:
            raise ValueError("API key not found in environment variables")
    if start_date is None:
        start_date = (datetime.now() - timedelta(days=7)).strftime('%Y%m%dT0000')
    else:
        start_date = datetime.strptime(start_date, '%Y-%m-%d').strftime('%Y%m%dT0000')
    if end_date is None:
        end_date = datetime.now().strftime('%Y%m%dT0000')
    else:
        end_date = datetime.strptime(end_date, '%Y-%m-%d').strftime('%Y%m%dT0000')

    news_data = []
    try:
        for symbol in symbols:
            url = f"https://www.alphavantage.co/query"
            params ={
                'function': 'NEWS_SENTIMENT',
                'tickers': symbol,
                'apikey': api_key,
                'time_from': start_date,
                'time_to': end_date,
                #'sort': 'LATEST',
                
            }
            response = requests.get(url, params=params)
            data = response.json()
            if 'feed' not in data:
                print(f"No news found for {symbol} in the given date range")
                continue
            for item in data['feed']:
               news = {
                   'symbol': symbol,
                   'title': item['title'],
                   'source': item['source'],
                   'date': datetime.strptime(item['time_published'], '%Y%m%dT%H%M%S'),
                   'summary': item['summary'],
                   'url': item['url'],
               }
               
               if 'ticker_sentiment' in item:
                   for ticker_data in item['ticker_sentiment']:
                       if ticker_data['ticker'] == symbol:
                           news['sentiment'] = ticker_data['ticker_sentiment_label']
                           news['sentiment_score'] = ticker_data['ticker_sentiment_score']
                           break

               news_data.append(news)
                
    except Exception as e:
        print(f"Error fetching news for {symbol}: {e}")
        return pd.DataFrame()
    
    if news_data:
        news_df = pd.DataFrame(news_data)
        news_df = news_df.sort_values(by='date', ascending=False)
        return news_df
    else:
        return pd.DataFrame()

    

                        


<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\zxmat\AppData\Local\Temp\ipykernel_34884\1673413519.py:1: SyntaxWarning: invalid escape sequence '\e'
  load_dotenv(dotenv_path='../erdos_finance_llm\erdos.env')


In [41]:
test_news = get_news(
    symbols=["TSLA",'AAPL','IBM','NFLX'],
    start_date="2025-03-04",
    end_date="2025-03-23",
    api_key=os.getenv('ALPHA_VANTAGE_API_KEY')
)

In [42]:
test_news

,symbol,title,source,date,summary,url,sentiment,sentiment_score
0,TSLA,"Down 42%, Should You Buy Tesla Stock on the Dip?",Motley Fool,2025-03-22 22:18:59,Tesla ( NASDAQ: TSLA ) stock looks like a fall...,https://www.fool.com/investing/2025/03/22/down...,Somewhat-Bullish,0.162567
50,AAPL,"The Best Warren Buffett Stocks to Buy With $2,...",Motley Fool,2025-03-22 20:13:00,The S&P 500 is down more than 7% over the past...,https://www.fool.com/investing/2025/03/22/the-...,Neutral,0.083047
128,NFLX,Could Netflix Stock Help You Retire a Milliona...,Motley Fool,2025-03-22 18:45:00,There are few businesses that have taken care ...,https://www.fool.com/investing/2025/03/22/coul...,Somewhat-Bullish,0.254421
51,AAPL,Should You Buy Berkshire Hathaway While It's B...,Motley Fool,2025-03-22 18:05:00,While the stock market has been selling off re...,https://www.fool.com/investing/2025/03/22/shou...,Neutral,0.04501
1,TSLA,Elon Musk Suggests Trans People at Fault for T...,Benzinga,2025-03-22 17:33:25,Elon Musk blames transgender people for Tesla ...,https://www.benzinga.com/markets/25/03/4444898...,Somewhat-Bearish,-0.275137
...,...,...,...,...,...,...,...,...
123,IBM,IBM Extends JNPR Tie-Up for AI-Native Networki...,Zacks Commentary,2025-03-06 14:40:00,Riding on a robust earnings surprise history a...,https://www.zacks.com/stock/news/2426655/ibm-e...,Bullish,0.48516
124,IBM,Should iShares MSCI USA Min Vol Factor ETF ( ...,Zacks Commentary,2025-03-06 11:20:07,Style Box ETF report for ...,https://www.zacks.com/stock/news/2426403/shoul...,Neutral,0.067432
125,IBM,Rigetti Claims 'Leadership' In Quantum Computi...,Benzinga,2025-03-06 07:18:04,Rigetti Computing Inc. RGTI missed its fourth-...,https://www.benzinga.com/general/market-summar...,Bullish,0.365676
126,IBM,4 Cloud Computing Stocks to Watch Amid AI Prol...,Zacks Commentary,2025-03-05 14:41:00,"IBM, MSFT, AMZN and GOOGL are some of the indi...",https://www.zacks.com/stock/news/2425900/4-clo...,Neutral,0.072745


In [31]:
NEWS_ANALYSIS_PROMPT = """
    You are an expert financial analyst.
    I will provide you with a news article related to a stock.
    1. Determine sentiment: Positive, Negative, Neutral.
    2. Summary key points relevant to the stock.
    3. Provide an investment recommendation based on this specific article.
    4. You must respond with valid JSON only


    Format your output as :
    {
        "sentiment": "Positive/Negative/Neutral",
        "key_points": [
            "Key point 1:",
            "key point 2:"
            
        ],
        
        "investment_recommendation": "Buy/Hold/Sell "
    }
    """

In [32]:
from groq import Groq
import json 
import re

In [33]:
import json
import pandas as pd

def analyze_news(news_df, api_key=None, model='llama3-8b-8192'):
    if api_key is None:
        raise ValueError("API key is required")
    client = Groq(api_key=api_key)
    
    # Ensure 'date' is a datetime column and create a date-only column.
    news_df['date_only'] = pd.to_datetime(news_df['date']).dt.date
    
    grouped_news = news_df.groupby(['symbol', 'date_only'])
    results = []
    
    for (symbol, date), group in grouped_news:
        print(f"Processing {symbol} on {date}")
        articles_text = '\n\n'.join(
            [f'Title: {row["title"]}\nContent: {row["summary"]}' for _, row in group.iterrows()]
        )
        
        try:
            completion = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": NEWS_ANALYSIS_PROMPT},
                    {"role": "user", "content": f"Stock: {symbol}\nDate: {date}\nNews Articles:\n{articles_text}"}
                ],
                model=model,
                temperature=0.1
            )
            analysis = completion.choices[0].message.content
            
            # Attempt to parse the LLM output as JSON.
            try:
                result = json.loads(analysis)
            except Exception as parse_err:
                cleaned_text = analysis.strip()
                if '{' in cleaned_text and '}' in cleaned_text:
                    json_candidate = cleaned_text[cleaned_text.find('{'): cleaned_text.rfind('}')+1]
                    try:
                        result = json.loads(json_candidate)
                    except Exception as inner_err:
                        print(f"Failed to parse JSON for {symbol} on {date}: {inner_err}")
                        result = {
                            "sentiment": "parsing_error",
                            "key_points": ["parsing_error"],
                            "investment_recommendation": "failed"
                        }
                else:
                    print(f"No JSON structure found in LLM output for {symbol} on {date}")
                    result = {
                        "sentiment": "parsing_error",
                        "key_points": ["parsing_error"],
                        "investment_recommendation": "failed"
                    }
            
            results.append({
                'symbol': symbol,
                'date': date,
                'news_count': len(group),
                'ai_sentiment': result.get('sentiment', ''),
                'ai_key_points': result.get('key_points', []),
                'ai_recommendation': result.get('investment_recommendation', '')
            })
        except Exception as e:
            print(f"Error processing {symbol} on {date}: {e}")
            results.append({
                'symbol': symbol,
                'date': date,
                'news_count': len(group),
                'ai_sentiment': 'processing_error',
                'ai_key_points': [f'processing_error: {str(e)}'],
                'ai_recommendation': f'processing_error: {str(e)}'
            })
                
    return pd.DataFrame(results)


In [43]:
analyze_news(test_news, api_key=os.getenv('GROQ_API_KEY'), model='llama3-8b-8192')

Processing AAPL on 2025-03-18
Processing AAPL on 2025-03-19
Processing AAPL on 2025-03-20
Processing AAPL on 2025-03-21
Processing AAPL on 2025-03-22
Processing IBM on 2025-03-05
Processing IBM on 2025-03-06
Processing IBM on 2025-03-10
Processing IBM on 2025-03-11
Processing IBM on 2025-03-12
Processing IBM on 2025-03-13
Processing IBM on 2025-03-14
Processing IBM on 2025-03-15
Processing IBM on 2025-03-17
Processing IBM on 2025-03-18
Processing IBM on 2025-03-19
Processing IBM on 2025-03-20
Processing IBM on 2025-03-21
Processing NFLX on 2025-03-12
Processing NFLX on 2025-03-13
Processing NFLX on 2025-03-14
Processing NFLX on 2025-03-15
Processing NFLX on 2025-03-16
Processing NFLX on 2025-03-17
Processing NFLX on 2025-03-18
Processing NFLX on 2025-03-19
Processing NFLX on 2025-03-20
Processing NFLX on 2025-03-21
Processing NFLX on 2025-03-22
Processing TSLA on 2025-03-21
Processing TSLA on 2025-03-22


,symbol,date,news_count,ai_sentiment,ai_key_points,ai_recommendation
0,AAPL,2025-03-18,2,Positive,[AAPL has outperformed the market over the pas...,Buy
1,AAPL,2025-03-19,14,Neutral,"[AAPL stock is mentioned in multiple articles,...",Hold
2,AAPL,2025-03-20,16,Neutral,"[AAPL closed at $214.10, marking a -0.53% move...",Hold
3,AAPL,2025-03-21,12,Negative,"[Warren Buffett is selling off Apple stock, Ap...",Sell
4,AAPL,2025-03-22,6,Neutral,[The S&P 500 is down more than 7% over the pas...,Hold
5,IBM,2025-03-05,2,Neutral,[IBM is mentioned as one of the indispensable ...,Hold
6,IBM,2025-03-06,4,Positive,[IBM extends its tie-up with JNPR for AI-nativ...,Buy
7,IBM,2025-03-10,2,Negative,[The semiconductor sector is experiencing a sl...,Hold
8,IBM,2025-03-11,3,Positive,[D-Wave Quantum's Q4 performance is expected t...,Buy
9,IBM,2025-03-12,1,Neutral,[The article does not specifically mention IBM...,Hold
